In [1]:
%matplotlib inline
# Imports
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import median
from skimage.feature import blob_log
from skimage.draw import circle
import os, math, csv
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import warnings
warnings.simplefilter("ignore")
# CSV file name
CSV = "Results.csv"
# Center of the detection circle
CENTER = (2020, 2020)
# Radius of the detection circle
RADIUS = 1750
# Minimal distance of the blobs from the circle border
BORDER_DISTANCE = 15
# Lower for smaller dots
MIN_SIZE = 1.5
# Higher for bigger dots
MAX_SIZE = 8
# Higher for increased precision
LAYERS = 15
# Lower to also detect darker dots
THRESHOLD = .015
# Allowed overlap for dots
OVERLAP = 0.35
# Blurring radius
BLURRING = (5, 5)
#---------------------
img_path = os.path.join(os.getcwd(), "Images")
# Load images
images = []
rows = []
for t in os.walk(img_path):
    for file in t[2]:
        images.append(os.path.join(t[0], file))

def analyse_image(path):
    # Load image
    image = rgb2gray(io.imread(path))
    # Calculate image average
    av = np.average(image)
    # Create mask filled with average
    mask = np.full(fill_value=av, shape=image.shape, dtype="uint8")
    # Mask image
    rr, cc = circle(CENTER[0], CENTER[1], RADIUS)
    mask[rr, cc] = image[rr, cc]
    # Create result image
    #fig = plt.figure(figsize=(10, 10))
    #ax = fig.add_subplot(1, 1, 1)
    #plt.imshow(mask, cmap="gray")

    def eu_dist(p1, p2):
        return math.sqrt(((p2[0] - p1[0])**2) + ((p2[1] - p1[1])**2))

    # Detect blobs
    blobs = blob_log(image=median(mask, selem=np.ones(shape=BLURRING)),
                     min_sigma=MIN_SIZE, max_sigma=MAX_SIZE,
                     num_sigma=LAYERS, threshold=THRESHOLD,
                     exclude_border=False, overlap=OVERLAP)
    count = 0
    # Draw blobs
    for blob in blobs:
        y, x, r = blob
        if eu_dist((x, y), CENTER) <= RADIUS - BORDER_DISTANCE:
            c = plt.Circle((x, y), r, color="red", linewidth=0.85, fill=False)
            ax.add_patch(c)
            count += 1
    #ax.set_title(f"Count: {count}")
    #plt.show()
    return path, count

with open(CSV, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=" ",
                        quotechar="|", quoting=csv.QUOTE_MINIMAL)
    writer.writerow(("Path", "Count"))
    for path in images:
        row = analyse_image(path)
        writer.writerow(row)